In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
#from webdriver_manager.chrome import ChromeDriverManager
import os
import json
import time
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
import re
import matplotlib.pyplot as plt
import pickle
import matplotlib

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--window-size=1920x1080')
chrome_options.headless = True
driver = webdriver.Chrome('chromedriver',options=chrome_options)

<ipython-input-16-a3b6e45ef493>:4: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


In [ ]:
driver.get("https://www.alibaba.com/trade/search?spm=a27aq.cp_43.6590955380.16.42ac38a1XUtYqu&CatId=100010631&SearchText=Agricultural+Machinery+%26+Equipment&language=en&indexArea=product_en")
driver.maximize_window()

In [ ]:
#//li[@class='styles-module-listItem-_dG7y styles-module-listItem_arrow-x_snQ styles-module-listItem_arrow_next-GnEQw']/a
#//div[@class='iva-item-content-rejJg']
#//div[@class='iva-item-content-rejJg']/div/div[@class='iva-item-titleStep-pdebR']
#//div[@class='iva-item-content-rejJg']/div/div[@class='iva-item-priceStep-uq2CQ']
#//div[@class='iva-item-content-rejJg']/div/div[@class='price-root-RA1pj']
# iva-item-descriptionStep-C0ty1
# //span[@class='price-number']
# //a[@class='pagination-pages-link']/i
with open('Agriculture.jsonl', 'w') as f:
  for _ in tqdm( range(30)):
    #titles = driver.find_elements(By.XPATH, "//div[@class='iva-item-content-rejJg']/div/div[@class='iva-item-titleStep-pdebR']")
    #print(titles)
    prices = driver.find_elements(By.XPATH, "//span[@class='elements-offer-price-normal__price']")
    #description = driver.find_elements(By.XPATH, "//div[@class='iva-item-content-rejJg']/div/div[@class='iva-item-descriptionStep-C0ty1']")
    for  price in prices:
      dic = {}
      #dic['title'] = title.text
      price = re.sub(r'\n', '', price.text)
      price = re.sub(r'\$', '', price)
      price = re.sub(r',', '', price)
      dic['price'] = price.strip()
      #dic['description'] = desc.text
      f.write(json.dumps(dic) + '\n')
    button = driver.find_element(By.XPATH, "//a[@class='seb-pagination__pages-link pages-next']/i")
    #time.wait(10)
    #button.click()

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
final = []
for line in open('electronic_ob.jsonl', 'r'):
  dic = {}
  get = json.loads(line)
  price = get['price'] 
  price = re.sub(r'\n', '', price)
  price = re.sub(r'\$', '', price)
  price = re.sub(r',', '.', price)
  dic['price'] = price.strip()
  final.append(dic)
with open('electronic_ob.jsonl', 'w') as f:
  for piece in final:
    f.write(json.dumps(piece) + '\n')

In [ ]:
final

[{'price': '7 300.00  - 7 500.00'},
 {'price': '80 000.00  - 98 000.00'},
 {'price': '22 000.00  - 76 800.00'},
 {'price': '23 000.00  - 76 900.00'},
 {'price': '11 300.00  - 11 500.00'},
 {'price': '2 499.00'},
 {'price': '40 000.00  - 45 000.00'},
 {'price': '80 000.00  - 90 000.00'},
 {'price': '1 000.00  - 3 000.00'},
 {'price': '11 300.00  - 11 500.00'},
 {'price': '80 000.00  - 100 000.00'},
 {'price': '13 000.00  - 20 000.00'},
 {'price': '0.19  -2.59'},
 {'price': '0.80  -1.20'},
 {'price': '75 000.00'},
 {'price': '4 700.00'},
 {'price': '7 439.00  - 7 499.00'},
 {'price': '14 000.00  - 20 000.00'},
 {'price': '75.00  -143.00'},
 {'price': '220.00  -240.00'},
 {'price': '0.52  -1.00'},
 {'price': '0.10  -3.00'},
 {'price': '0.10  -10.00'},
 {'price': '11 500.00'},
 {'price': '1.20  -10.00'},
 {'price': '98 000.00  - 180 000.00'},
 {'price': '0.50  -1.30'},
 {'price': '0.40  -2.10'},
 {'price': '0.30  -0.50'},
 {'price': '0.50  -1.00'},
 {'price': '0.10  -5.00'},
 {'price': '0.

In [ ]:
import requests, json, sys

key = 'af0deccbgcgidddjgnvljitntccdduijhdinfgjgfjir' # ключ, с которым всё работает, не разбирался где его брать, но похоже он статичен, т.к. гуглится на различных форумах
cookie = '__cfduid=da6b6b5b9f01fd022f219ed53ac3935791610912291; sessid=ef757cc130c5cd228be88e869369c654.1610912291; _ga=GA1.2.559434019.1610912292; _gid=GA1.2.381990959.1610912292; _fbp=fb.1.1610912292358.1831979940; u=2oiycodt.1oaavs8.dyu0a4x7fxw0; v=1610912321; buyer_laas_location=641780; buyer_location_id=641780; luri=novosibirsk; buyer_selected_search_radius4=0_general; buyer_local_priority_v2=0; sx=H4sIAAAAAAACAxXLQQqAIBAF0Lv8dYvRLEdvU0MIBU0iKCHePXr71zGfefd1W5RLYick2kSakiB2VETclpf85n19RJMSp4vJOSlM%2F2BMOBDNaigE9taM8QH0oydNVAAAAA%3D%3D; dfp_group=100; _ym_uid=1610912323905107257; _ym_d=1610912323; _ym_visorc_34241905=b; _ym_isad=2; _ym_visorc_419506=w; _ym_visorc_188382=w; __gads=ID=2cff056a4e50a953-22d0341a94b900a6:T=1610912323:S=ALNI_MZMbOe0285QjW7EVvsYtSa-RA_Vpg; f=5.8696cbce96d2947c36b4dd61b04726f1a816010d61a371dda816010d61a371dda816010d61a371dda816010d61a371ddbb0992c943830ce0bb0992c943830ce0bb0992c943830ce0a816010d61a371dd2668c76b1faaa358c08fe24d747f54dc0df103df0c26013a0df103df0c26013a2ebf3cb6fd35a0ac0df103df0c26013a8b1472fe2f9ba6b978e38434be2a23fac7b9c4258fe3658d831064c92d93c3903815369ae2d1a81d04dbcad294c152cb0df103df0c26013a20f3d16ad0b1c5462da10fb74cac1eab2da10fb74cac1eab3c02ea8f64acc0bdf0c77052689da50d2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab91e52da22a560f5503c77801b122405c48ab0bfc8423929a6d7a5083cc1669877def5708993e2ca678f1dc04f891d61e35b0929bad7c1ea5dec762b46b6afe81f200c638bc3d18ce60768b50dd5e12c30e37135e8f7c6b64dc9f90003c0354a346b8ae4e81acb9fa46b8ae4e81acb9fa02c68186b443a7acf8b817f3dc0c3f21c1eac53cc61955882da10fb74cac1eab2da10fb74cac1eab5e5aa47e7d07c0f95e1e792141febc9cb841da6c7dc79d0b' \
# Если забанили, то добавьте свои куки, это не боевой код но он делает то, что надо
search = 'станок+для+производства'     # Строка поиска на сайте и ниже параметры выбора города, радиуса разброса цены и т.п.
categoryId = 14
locationId = 641780         # Новосибирск
searchRadius = 200
priceMin = 0
priceMax = 2000000
sort = 'priceDesc'
withImagesOnly = 'true'     # Только с фото
limit_page = 50     # Количество объявлений на странице 50 максимум

def except_error(res): # Эту функцию можно дополнить, например обработку капчи
    print(res.status_code, res.text)
    sys.exit(1)

s = requests.Session()                          # Будем всё делать в рамках одной сессии
# Задаем заголовки:
headers = { 'authority': 'm.avito.ru',
            'pragma': 'no-cache',
            'cache-control': 'no-cache',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Mobile Safari/537.36',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'sec-fetch-site': 'none',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-user': '?1',
            'sec-fetch-dest': 'document',
            'accept-language': 'ru-RU,ru;q=0.9',}
if cookie:                                      # Добавим куки, если есть внешние куки
    headers['cookie'] = cookie
s.headers.update(headers)                       # Сохраняем заголовки в сессию
s.get('https://m.avito.ru/')                    # Делаем запрос на мобильную версию.
url_api_9 = 'https://m.avito.ru/api/9/items'    # Урл первого API, позволяет получить id и url объявлений по заданным фильтрам
                                                # Тут уже видно цену и название объявлений
params = {
    'categoryId': 14,
    'params[30]': 4969,
    'locationId': locationId,
    'searchRadius': searchRadius,
    'priceMin': priceMin,
    'priceMax': priceMax,
    'params[110275]': 426645,
    'sort': sort,
    'withImagesOnly': withImagesOnly,
    'lastStamp': 1610905380,
    'display': 'list',
    'limit': limit_page,
    'query': search,
}
cicle_stop = True       # Переменная для остановки цикла
cikle = 0               # Переменная для перебора страниц с объявлениями
items = []              # Список, куда складываем объявления
params['key'] =  key
while cicle_stop:
    cikle += 1          # Так как страницы начинаются с 1, то сразу же итерируем
    params['page'] = cikle
    res = s.get(url_api_9, params=params)
    try:
        res = res.json()
    except json.decoder.JSONDecodeError:
        except_error(res)
    if res['status'] != 'ok':
            print(res['result'])
            sys.exit(1)
    if res['status'] == 'ok':
        items_page = int(len(res['result']['items']))

        if items_page > limit_page: # проверка на "snippet"
            items_page = items_page - 1

        for item in res['result']['items']:
            if item['type'] == 'item':
                items.append(item)
        if items_page < limit_page:
            cicle_stop = False
####################################################################
params = {'key': key}
for i in items: # Теперь идем по ябъявлениям:
    ad_id = str(i['value']['id'])
    # url_more_data_1 = 'https://m.avito.ru/api/1/rmp/show/' + ad_id  # more_data_1 = s.get(url_more_data_1, params=params).json() # Тут тоже моного информации, можете посмотреть
    url_more_data_2 = 'https://m.avito.ru/api/15/items/' + ad_id
    more_data_2 = s.get(url_more_data_2, params=params).json()
    if not 'error' in more_data_2:
        # print(more_data_2)            # В more_data_2 есть всё, что надо, я вывел на принт наиболее интересные для наглядности:
        print(more_data_2['title'])
        print(more_data_2['price'])
        print(more_data_2['address'])
        url_get_phone = 'https://m.avito.ru/api/1/items/' + ad_id + '/phone'    # URL для получения телефона
        phone = s.get(url_get_phone, params=params).json()                      # Сам запрос
        if phone['status'] == 'ok': phone_number = requests.utils.unquote(phone['result']['action']['uri'].split('number=')[1]) # Прверка на наличие телефона, такой странный синтсксис, чтоб уместиться в 100 сторочек кода)))
        else: phone_number = phone['result']['message']
        print(phone_number)
        print(more_data_2['seller'])
        # print(more_data_2['description']) # Скрыл, т.к. много букв
        print('=======================================================\n')

{'message': 'Доступ с вашего IP-адреса временно ограничен', 'link': 'ru.avito://1/firewall/captcha/show'}


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
